In [ ]:
import tweepy
import pandas as pd
import numpy as np

In [ ]:
import sys
sys.path

In [ ]:
sys.path.append('D:\\Analytics\\Python ML\\Twitter Data')

In [ ]:
sys.path

In [ ]:
from api_keys import *

In [ ]:
# API's setup:
def twitter_setup():
    """
    Utility function to setup the Twitter's API
    with our access keys provided.
    """
    # Authentication and access using keys:
    auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)

    # Return API with authentication:
    api = tweepy.API(auth,wait_on_rate_limit=True)
    return api

In [ ]:
#In list1 - add associated multiple hashtags
list1=[]
Hashtag_tweets = {}

In [ ]:
# We create an extractor object:
extractor = twitter_setup()
for hashtag in list1:
    tweets = [tweet for tweet in tweepy.Cursor(extractor.search,q=str(hashtag),lan="en").items(10)]
    print(len(tweets))
    Hashtag_tweets.update({hashtag:tweets})

In [ ]:
Hashtag_tweets

In [ ]:
for key,value in Hashtag_tweets.items():
    for v in value:
        print(key+': '+v.text)
    print('\n')

In [ ]:
data = pd.DataFrame([[key,v.text] for key,value in Hashtag_tweets.items() for v in value], columns=['Hashtags','Tweets'])
data

In [ ]:
#Adding new columns in the dataframe
# We add relevant data: Hashtag_tweets
data['Len'] = [len(v.text) for key,value in Hashtag_tweets.items() for v in value]
data

In [ ]:
data['ID'] = [v.id for key,value in Hashtag_tweets.items() for v in value]
data['Date'] = [v.created_at for key,value in Hashtag_tweets.items() for v in value]
data['Sources'] = [v.source for key,value in Hashtag_tweets.items() for v in value]
data['Likes'] = [v.favorite_count for key,value in Hashtag_tweets.items() for v in value]
data['Retweets'] = [v.retweet_count for key,value in Hashtag_tweets.items() for v in value]
data

In [ ]:
data.groupby('Hashtags', as_index=False).mean()

In [ ]:
from textblob import TextBlob
import re

In [ ]:
def clean_tweet(tweet):
    '''
    Utility function to clean the text in a tweet by removing 
    links and special characters using regex.
    '''
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

In [ ]:
def analyze_sentiment(tweet):
    '''
    Utility function to classify the polarity of a tweet
    using textblob.
    '''
    analysis = TextBlob(clean_tweet(tweet))
    if analysis.sentiment.polarity > 0:
        return 1 #+ve polarity => +ve characteristics
    elif analysis.sentiment.polarity == 0:
        return 0 #neutral polarity => neutral characteristics
    else:
        return -1 #-ve polarity => -ve characteristics

In [ ]:
#data['Sentiment-Analysis'] = [analyze_sentiment(v.text) for key,value in Hashtag_tweets.items() for v in value]
data['Sentiment-Analysis'] = [analyze_sentiment(tweet) for tweet in data['Tweets']]
data

In [ ]:
# We construct lists with classified tweets:
pos_tweets = [tweet for index,tweet in enumerate(data['Tweets']) if data['Sentiment-Analysis'][index] > 0]
neu_tweets = [tweet for index,tweet in enumerate(data['Tweets']) if data['Sentiment-Analysis'][index] == 0]
neg_tweets = [tweet for index,tweet in enumerate(data['Tweets']) if data['Sentiment-Analysis'][index] < 0]

In [ ]:
# We print percentages:
print('Percentage of Positive Tweets: ',int(len(pos_tweets))*100/int(len(data['Tweets'])))
print('Percentage of Neutral Tweets: ',int(len(neu_tweets))*100/int(len(data['Tweets'])))
print('Percentage of Negative Tweets: ',int(len(neg_tweets))*100/int(len(data['Tweets'])))